# Tutorial for search EP4 workflow data via digitaltwins-on-fhir
Install package


*   pip install digitaltwins-on-fhir



In [ ]:
pip install digitaltwins-on-fhir

In [ ]:
from digitaltwin_on_fhir import Adapter
from pprint import pprint
adapter = Adapter("http://130.216.217.173:8080/fhir")
client = adapter.async_client

# Example 1 Find the result observation is generated by which workflow and related to which dataset

TODO 1 I'm interested in a result observation, e.g, distance to nipple

And the uuid is: sparc-workflow-tool-clinical_report-777-workflow-tool-7-process-002-Observation-0



*   Let's find which workflow / wolkflow tool generate this observation
*   Let's find the observation related to which dataset


In [ ]:
ob = await client.resources("Observation").search(
            identifier="sparc-workflow-tool-clinical_report-777-workflow-tool-7-process-002-Observation-0").first()
print("Step 0: We are interested in this observation (Tumour distance to nipple): ", ob)

Let's find the observation is generated by which workflow

In [ ]:
# step 1, Find this observation is assigned in which workflow result composition
composition = await client.resources("Composition").search(title="workflow tool results",
                                                            entry=ob.to_reference().reference).first()
print("Step 1: the interested observation is assigned in this composition: ", composition)
# step 2, Let's find which workflow tool process generates the composition
workflow_tool_process = await composition.get("subject").to_resource()
print("Step 2: the composition is generated by this workflow_tool_process: ", workflow_tool_process)
# step 3, Now we can find the observation is related to which workflow
workflow = await workflow_tool_process.get("for").to_resource()
print("Step 3: Now we find the interested observation is related to this workflow: ", workflow)
# step 3.1, Because we know the interested observation, so we can directly find which workflow tool generates it
workflow_tool = await ob.get("focus")[0].to_resource()
print("Step 3.1: We also can find the interested observation is related to this workflow_tool: ", workflow_tool)

Let's also find the interested observation is related which dataset

In [ ]:
# step 4, let's find which research subject is related to this observation
research_subject = await workflow_tool_process.get("basedOn")[0].to_resource()
print("Step 4, the interested observation is related to this research_subject: ", research_subject)
# step 4.1, we also can find this observation is generated by which patient
patient = await composition.get("author")[0].to_resource()
print("Step 4.1: the observation comes from this patient: ", patient)
# step 5, now we can know the observation is related to which dataset
dataset = await research_subject.get("study").to_resource()
print("Step 5, Now we find the interested observation is related to this dataset: ", dataset)

# Example 2 Find all workflow tools and workflow tool processes related to the workflow

TODO 2 If we know the workflow uuid, let's find the workflow

TODO 2.1 let's find all related workflow tools

In [ ]:
workflow = await client.resources("PlanDefinition").search(identifier="sparc-workflow-uuid-001").first()
actions = workflow.get("action")
for a in actions:
  if a.get("definitionCanonical") is None:
      continue
  resource_type, _id = a.get("definitionCanonical").split("/")
  workflow_tool = await client.reference(resource_type, _id).to_resource()
  print(f"workflow tool: {workflow_tool.get('title')}", workflow_tool)
  # TODO 2.1.1 Find model and software uuid from a specific workflow tool
  print("workflow tool software and model uuid", workflow_tool.get("participant"))

TODO 2.2 let's find all related workflow tool process for that workflow

In [ ]:
workflow_tool_processes = await client.resources("Task").search(
            subject=workflow.to_reference().reference).fetch_all()
print(workflow_tool_processes)

# Example 3 Find a specific workflow tool process for a patient in a dataset
This requires:

*   patient uuid,
*   workflow uuid,
*   workflow tool uuid,
*   and dataset uuid



In [ ]:
# Step 1: find the patient
patient = await client.resources("Patient").search(identifier="sparc-breast-patient-001").first()
# Step 2: find the dataset
dataset = await client.resources("ResearchStudy").search(identifier="sparc-breast-dataset-001").first()
# Step 3: find the workflow tool
workflow_tool = await client.resources("ActivityDefinition").search(
    identifier="sparc-workflow-tool-clinical_report-777").first()
# Step 4: find the research subject
research_subject = await client.resources("ResearchSubject").search(patient=patient.to_reference().reference,
                                                                    study=dataset.to_reference().reference).first()
print("research_subject: ", research_subject)
workflow_tool_process = await client.resources("Task").search(subject=workflow.to_reference(),
                                                              focus=workflow_tool.to_reference(),
                                                              based_on=research_subject.to_reference(),
                                                              owner=patient.to_reference()).first()
print("Now the workflow_tool_process is: ", workflow_tool_process)

Now we get the breast workflow's step 7 workflow tool's workflow tool process which belongs to patient 1 in dataset 1

TODO 3.1 Get the input of the workflow tool process

In [ ]:
inputs = workflow_tool_process.get("input")
for i in inputs:
    composition = await client.resources("Composition").search(title="primary measurements",
                                                                entry=i.get("valueReference")).first()
    print("primary measurement's composition: ", composition)
    dataset = await composition.get("subject").to_resource()
    print("dataset: ", dataset)

TODO 3.2 We also can list all outputs of the workflow tool process

In [ ]:
outputs = workflow_tool_process.get("output")
pprint(outputs)

TODO 3.3 We also can list all output resources

In [ ]:
for o in outputs:
  output_resource = await o.get("valueReference").to_resource()
  print(output_resource)